# Global Minimial Viable Products (1)

Combine:
* Outcrop Permiability
* Global Lithology
* MERIT terrain slope
* ERA5 Runoff
* USDA Soil Texture
* Land Use

into honduras product for water restoration

In [1]:
import ee
import numpy as np

import sys
sys.path.insert(1, '../datasets/SMAP-USDA/')  # until we package this
from earthshot import water_viz as vis
from earthshot import water_common as common
from earthshot import normalize as norm
from earthshot import mon_stats
from statistics import mean
import folium
from folium import plugins
import geemap.eefolium as geemap

Loading BokehJS ...

In [2]:
#ee.Authenticate()
ee.Initialize()

In [3]:
#define global box
bbox_global = ee.Geometry.BBox(-180,-90,180,90)
bbox_honduras = ee.Geometry.BBox(-86.43, 13.76, -83.73, 16.06)


# Outcrop Permiability

In [4]:
Permiability_1 = ee.FeatureCollection('users/amgadellaboudy/Permiability_Data')

perm_img = Permiability_1.reduceToImage(['logK_Ice_x'], ee.Reducer.mean())
perm_scaled = norm.img_scale(perm_img, area_of_interest = bbox_honduras)

In [5]:
print(perm_scaled.bandNames().getInfo())

['mean']


In [6]:
perm = perm_scaled.expression(
        "(b('mean') > 0.66) ? 3" +
        ": (b('mean') <= 0.66 && b('mean') > 0.33) ? 2" +
        ": (b('mean') <= 0.33 && b('mean') > 0.01) ? 1"
        ": 0").clip(bbox_honduras).updateMask(perm_scaled.lte(0.95))

## Lithology

In [7]:
Lithology = ee.Image('users/amgadellaboudy/Honduras_Lithology_2').select(['b1','b2','b3']).clip(bbox_honduras)
Lithology_reduced = Lithology.reduce(ee.Reducer.stdDev())

litho_mask = Lithology_reduced.lte(5)
litho_img = ee.Image('users/amgadellaboudy/Honduras_Lithology_2').select(['b1']).clip(bbox_honduras).updateMask(litho_mask)


In [8]:
litho = litho_img.expression(
        "(b('b1') > 0 && b('b1') <= 34) ? 0" +       #Undefined/ glaciers/ ice
        ":(b('b1') > 34 && b('b1') <= 51) ? 2" +      #Metamorphic
        ":(b('b1') > 51 && b('b1') <= 102) ? 2" +     #Intrusive
        ":(b('b1') > 102 && b('b1') <= 153) ? 2" +    #Volcanic
        ":(b('b1') > 153 && b('b1') <= 170) ? 0" +    #Evaporitic
        ":(b('b1') > 170 && b('b1') <= 187) ? 2" +   #Karsitic
        ":(b('b1') > 187 && b('b1') <= 204) ? 3" +   #Detritic
        ":(b('b1') > 204 && b('b1') <= 221) ? 2" +   #Pyroclastic (Volcanic)
        ":(b('b1') > 221 && b('b1') <= 238) ? 3" +   #Detritic
        ":(b('b1') > 238 && b('b1') <= 255) ? 3" +   #Alluvial
        ": 0").clip(bbox_honduras)

## ERA5 Runoff

In [9]:
# Sum up runoff across the year to include in product score
runoff_clim = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select('surface_runoff')

runoff_clim_m = mon_stats.bands_avgs(['surface_runoff'], runoff_clim)

runoff_img = ee.ImageCollection(runoff_clim_m['avgs'].get('surface_runoff')).sum().multiply(720)


runoff = runoff_img.expression(
        "(b('surface_runoff') > 5.833 ) ? 1" +
        ": (b('surface_runoff') <= 5.833 && b('surface_runoff') > 3.167) ? 2" +
        ": (b('surface_runoff') <= 3.167 && b('surface_runoff') > 0.496) ? 3" +
        ": 0").clip(bbox_honduras)

total_runoff = runoff_img.reduceRegion(reducer = ee.Reducer.sum(), geometry = bbox_honduras, scale = 100)
total_runoff_m = total_runoff.getInfo().get('surface_runoff')*0.000081

In [10]:
area = bbox_honduras.area(10).getInfo()
total_runoff_m3 = total_runoff_m*area
print(area)
print(total_runoff_m3)

74192432744.52104
131674432834802.6


## MERIT Terrain Slope

In [11]:
#include slope as a factor in scoring product, mask for lower slopes
slope_img = ee.Image('users/jamesmcc/merit_slope/merit_terrain_slope').clip(bbox_honduras)

slope_mask = slope_img.lte(21.8)

slope = slope_img.expression(
        "(b('slope') <= 21.8 && b('slope') > 14.533) ? 1" +
        ": (b('slope') <= 14.533 && b('slope') > 7.266) ? 2" +
        ": (b('slope') <= 7.266 && b('slope') >= 0) ? 3" +
        ": 0").clip(bbox_honduras)



In [12]:
print(slope_img.bandNames().getInfo())

['slope']


# Soil Types

In [13]:
#incorporate soil types as a factor in scoring for PRZ
soil_types = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")

In [14]:
#categorizing soil types and depths based on retaining plant-available water, grouping top soils and bottom soils together
soil_0 = soil_types.expression(
        "(b('b0') == 12) ? 3" +
        ": (b('b0') == 11) ? 3" +
        ": (b('b0') == 10) ? 3" +
        ": (b('b0') == 9) ? 3" +
        ": (b('b0') == 8) ? 2" +
        ": (b('b0') == 7) ? 2" +
        ": (b('b0') == 6) ? 2" +
        ": (b('b0') == 5) ? 2" +
        ": (b('b0') == 4) ? 1" +
        ": (b('b0') == 3) ? 1" +
        ": (b('b0') == 2) ? 1" +
        ": (b('b0') == 1) ? 1" +
        ": 0").clip(bbox_honduras)


                  

## Land Use

In [15]:
#https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global#image-properties
land_use = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('discrete_classification')
land_use_mask = land_use.neq(50).And(land_use.neq(90)).And(land_use.neq(80))

In [16]:
score = (perm.multiply(.215)
        .add(litho.multiply(.215))
        .add(slope.multiply(.21))
        .add(soil_0.multiply(.21))
        .add(runoff.multiply(.07)))

score_scaled = norm.img_scale(score, area_of_interest= bbox_honduras).updateMask(slope_mask).updateMask(land_use_mask)

In [17]:
honduras_land = ee.FeatureCollection('users/tsongkapa/HND-fo2_15')

In [18]:
viz_params_litho = {'min': 0, 'max': 3, 'dimensions': 512,
  'palette': ['White', 'Red', 'Yellow', 'Blue']}
viz_params_perm = {'min': 0, 'max': 3, 'dimensions': 512,
  'palette': ['White','Red', 'Yellow', 'Blue']}
viz_params_score = {'min': 0, 'max': 1, 'dimensions': 512,
  'palette': ['White','Red', 'Yellow', 'Blue']}

import branca
colormap = branca.colormap.LinearColormap(colors = ['Red', 'Yellow', 'Blue'], vmin = 0, vmax =1)
colormap = colormap.to_step(n=10)
colormap.caption = 'Score'



the_map = geemap.Map()
colormap.add_to(the_map)
#the_map.addLayer(litho, viz_params_litho, name = 'Litho')
the_map.addLayer(perm, viz_params_perm, name = 'Permeability')
the_map.addLayer(score_scaled, viz_params_score, name = 'Score')
the_map.addLayer(honduras_land.draw('#000000', strokeWidth = 1), {}, name = 'Honduras Land', opacity = 0.5)
vis.folium_display(the_map)

In [21]:
xx = ee.batch.Export.image.toAsset(score_scaled,
                                   assetId= 'users/amgadellaboudy/Honduras_Water_Score',
                                   scale = 90,
                                   description = 'Image_Task' ).start()